# Data cleaning

## Importação de bibliotecas e tabelas

In [223]:
#Importaremos as bibliotecas do pandas e numpy.
import pandas as pd
import numpy as np
import re

In [224]:
#Lendo a tabela ACCOUNTS.
account = pd.read_csv("C:/Users/pfaprado/Documents/IronHack/2021-01-29-project-financial-dataset/csvs/account.csv") 

In [225]:
#Lendo a tabela CREDITCARD.
creditcard = pd.read_csv("C:/Users/pfaprado/Documents/IronHack/2021-01-29-project-financial-dataset/csvs/card.csv")  

In [226]:
#Lendo a tabela CLIENTES.
client = pd.read_csv("C:/Users/pfaprado/Documents/IronHack/2021-01-29-project-financial-dataset/csvs/client.csv")  

In [227]:
#Lendo a tabela DISPOSITION.
disposition = pd.read_csv("C:/Users/pfaprado/Documents/IronHack/2021-01-29-project-financial-dataset/csvs/disp.csv")  

In [228]:
#Lendo a tabela DEMOGRAPH.
demograph = pd.read_csv("C:/Users/pfaprado/Documents/IronHack/2021-01-29-project-financial-dataset/csvs/district.csv")  

In [229]:
#Lendo a tabela TRANSACTIONS.
transactions = pd.read_csv("C:/Users/pfaprado/Documents/IronHack/2021-01-29-project-financial-dataset/csvs/trans.csv", sep =',')  

c:\users\pfaprado\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [230]:
"""-- As tabelas utilizadas serão:
-- coluna "operation" , tabela "transactions" -> queremos a soma de transações do tipo "VYBER KARTOU" / "credit card withdraw" por account_id
-- coluna "type", tabela "creditcard" -> queremos agrupar por tipos de cartões
-- coluna "region"/A3, tabela "demograph" -> queremos agrupar por regiões
-- necessitaremos a tabela "account", que conecta tabelas "demograph" e "transactions"
-- necessitaremos a tabela "disposition", que conecta tabelas "account" e "credit card" """

'-- As tabelas utilizadas serão:\n-- coluna "operation" , tabela "transactions" -> queremos a soma de transações do tipo "VYBER KARTOU" / "credit card withdraw" por account_id\n-- coluna "type", tabela "creditcard" -> queremos agrupar por tipos de cartões\n-- coluna "region"/A3, tabela "demograph" -> queremos agrupar por regiões\n-- necessitaremos a tabela "account", que conecta tabelas "demograph" e "transactions"\n-- necessitaremos a tabela "disposition", que conecta tabelas "account" e "credit card" '

## Tabela TRANSACTIONS

As colunas OPERATION, ACCOUNT_ID e TRANS_ID serão necessárias.
OPERATION possui dados nulos.
ACCOUNT_ID e TRANS_ID já estão organizadas.

In [231]:
transactions.info()
#Somente colunas OPERATION, K_SYMBOL, BANK e ACCOUNT tem dados nulos. Só trabalharemos a OPERATIONS.
#OPERATION é do tipo object/string.
#ACCOUNT_ID é do tipo int.
#TRANS_ID é do tipo int.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056320 entries, 0 to 1056319
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1056320 non-null  int64  
 1   trans_id    1056320 non-null  int64  
 2   account_id  1056320 non-null  int64  
 3   date        1056320 non-null  int64  
 4   type        1056320 non-null  object 
 5   operation   873206 non-null   object 
 6   amount      1056320 non-null  float64
 7   balance     1056320 non-null  float64
 8   k_symbol    574439 non-null   object 
 9   bank        273508 non-null   object 
 10  account     295389 non-null   float64
dtypes: float64(3), int64(4), object(4)
memory usage: 88.7+ MB


In [182]:
#Como não utilizaremos e não serão relevantes para a análise as colunas K_SYMBOL, BANK e ACCOUNT,
#vamos dropar las.
drop_cols_transactions = list(null_cols[null_cols > 300000].index)
transactions = transactions.drop(drop_cols_transactions, axis=1)
transactions.info()
#Ótimo, agora só temos a coluna operation com valores nulos.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1056320 entries, 0 to 1056319
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1056320 non-null  int64  
 1   trans_id    1056320 non-null  int64  
 2   account_id  1056320 non-null  int64  
 3   date        1056320 non-null  int64  
 4   type        1056320 non-null  object 
 5   operation   1056320 non-null  object 
 6   amount      1056320 non-null  float64
 7   balance     1056320 non-null  float64
dtypes: float64(2), int64(4), object(2)
memory usage: 72.5+ MB


### OPERATION

In [183]:
null_cols = transactions.isnull().sum()
null_cols
# OPERATION tem aproximadamente 10% dos dados faltando (183.114 de 1.056.320).
# OPERATION é a coluna na qual tem a categoria "credit card withdraw", de nosso interesse. Seria bom
# verificar o que está ocorrendo com esses dados.

Unnamed: 0    0
trans_id      0
account_id    0
date          0
type          0
operation     0
amount        0
balance       0
dtype: int64

In [184]:
#Verificando todos os valores existentes na coluna OPERATION.
set(transactions['operation'])

{'collection from another bank',
 'credit card withdrawal',
 'credit in cash',
 'other credit transactions',
 'remittance to another bank',
 'withdrawal in cash'}

In [185]:
operation_null = transactions[transactions['operation'].isnull()]
operation_null
#Aparentemente vemos "por cima" que aparecem nulos quando TYPE == 'PRIJEM'

,Unnamed: 0,trans_id,account_id,date,type,operation,amount,balance


In [186]:
#Verificando em uma subtabela o set de elementos presentes em TYPE na condição de OPERATION = Null:
set(operation_null[operation_null['type'] == "PRIJEM"]['type'])
#Portanto só existem nulos no OPERATION quando TYPE = 'PRIJEM'

set()

In [187]:
type_prijem = transactions[transactions['type']== "PRIJEM"]
set(type_prijem['operation'])
#Checando a coluna OPERATION quando TYPE é 'PRIJEM' (credit), é possível notar
#que só é considerado crédito 'prevod z uctu' ("COLLECTION FROM ANOTHER BANK")
#e 'VKLAD' ("withdrawl in cash").

{'collection from another bank', 'credit in cash', 'other credit transactions'}

In [188]:
#Vamos preencher esses nulos com "other credit transactions"
transactions['operation'] = transactions['operation'].fillna('other credit transactions')
#A cara do dataframe ficará:
transactions

,Unnamed: 0,trans_id,account_id,date,type,operation,amount,balance
0,0,695247,2378,930101,PRIJEM,credit in cash,700.0,700.0
1,1,171812,576,930101,PRIJEM,credit in cash,900.0,900.0
2,2,207264,704,930101,PRIJEM,credit in cash,1000.0,1000.0
3,3,1117247,3818,930101,PRIJEM,credit in cash,600.0,600.0
4,4,579373,1972,930102,PRIJEM,credit in cash,400.0,400.0
...,...,...,...,...,...,...,...,...
1056315,1056315,3626622,2906,981231,PRIJEM,other credit transactions,62.3,13729.4
1056316,1056316,3627616,2935,981231,PRIJEM,other credit transactions,81.3,19544.9
1056317,1056317,3625403,2869,981231,PRIJEM,other credit transactions,60.2,14638.2
1056318,1056318,3626683,2907,981231,PRIJEM,other credit transactions,107.5,23453.0


In [189]:
#Agora que substituímos os nulos, já estamos prontos para renomear os dados presentes na coluna OPERATION:

transactions['operation'] = transactions['operation'].replace(to_replace ="VYBER KARTOU", 
                 value ="credit card withdrawal")
transactions['operation'] = transactions['operation'].replace(to_replace ="VKLAD", 
                 value ="credit in cash")
transactions['operation'] = transactions['operation'].replace(to_replace ="PREVOD Z UCTU", 
                 value ="collection from another bank")
transactions['operation'] = transactions['operation'].replace(to_replace ="VYBER", 
                 value ="withdrawal in cash")
transactions['operation'] = transactions['operation'].replace(to_replace ="PREVOD NA UCET", 
                 value ="remittance to another bank")

#A cara do dataframe ficará:
transactions

,Unnamed: 0,trans_id,account_id,date,type,operation,amount,balance
0,0,695247,2378,930101,PRIJEM,credit in cash,700.0,700.0
1,1,171812,576,930101,PRIJEM,credit in cash,900.0,900.0
2,2,207264,704,930101,PRIJEM,credit in cash,1000.0,1000.0
3,3,1117247,3818,930101,PRIJEM,credit in cash,600.0,600.0
4,4,579373,1972,930102,PRIJEM,credit in cash,400.0,400.0
...,...,...,...,...,...,...,...,...
1056315,1056315,3626622,2906,981231,PRIJEM,other credit transactions,62.3,13729.4
1056316,1056316,3627616,2935,981231,PRIJEM,other credit transactions,81.3,19544.9
1056317,1056317,3625403,2869,981231,PRIJEM,other credit transactions,60.2,14638.2
1056318,1056318,3626683,2907,981231,PRIJEM,other credit transactions,107.5,23453.0


In [190]:
#Dropando as linhas duplicadas:
before = len(transactions)
transactions = transactions.drop_duplicates()
after = len(transactions)
print('Number of duplicate records dropped: ', str(before - after))
#Não possuímos linhas duplicadas nessa tabela TRANSACTIONS.

Number of duplicate records dropped:  0


In [191]:
#Por fim a nossa tabela TRANSACTIONS ficará sem nulos e com categorias do OPERATION traduzida.
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1056320 entries, 0 to 1056319
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1056320 non-null  int64  
 1   trans_id    1056320 non-null  int64  
 2   account_id  1056320 non-null  int64  
 3   date        1056320 non-null  int64  
 4   type        1056320 non-null  object 
 5   operation   1056320 non-null  object 
 6   amount      1056320 non-null  float64
 7   balance     1056320 non-null  float64
dtypes: float64(2), int64(4), object(2)
memory usage: 72.5+ MB


In [192]:
#Vamos exportar o dataframe na forma de CSV.
transactions.to_csv(r'C:/Users\pfaprado/Documents\IronHack/2021-01-29-project-financial-dataset/csvs_corrected_on_python/transactions_corrected.csv', sep = ',', index = False)

## Tabela DISPOSITION
As colunas DISP_ID e CLIENT_ID serão necessárias.

In [193]:
disposition.info()
#Sem dados nulos na tabela.
#CLIENT_ID é do tipo int.
#DISP_ID é do tipo int.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5369 entries, 0 to 5368
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5369 non-null   int64 
 1   disp_id     5369 non-null   int64 
 2   client_id   5369 non-null   int64 
 3   account_id  5369 non-null   int64 
 4   type        5369 non-null   object
dtypes: int64(4), object(1)
memory usage: 251.7+ KB


In [194]:
#Vamos retirar as linhas duplicadas:
before = len(disposition)
disposition = disposition.drop_duplicates()
after = len(disposition)
print('Number of duplicate records dropped: ', str(before - after))
#Não possuímos linhas duplicadas nessa tabela DISPOSITION.

Number of duplicate records dropped:  0


In [195]:
#disp_ID é a coluna de interesse nosso, pois será a Primary Key. A Foreign Key será disp_id da tabela
#creditcard.

## Tabela CREDITCARD
As colunas CARD_ID, DISP_ID e TYPE serão necessárias.

In [196]:
creditcard.info()
#Sem dados nulos na tabela.
#CARD_ID é do tipo int.
#DISP_ID é do tipo int.
#TYPE é do tipo object/string.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892 entries, 0 to 891
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  892 non-null    int64 
 1   card_id     892 non-null    int64 
 2   disp_id     892 non-null    int64 
 3   type        892 non-null    object
 4   issued      892 non-null    object
dtypes: int64(3), object(2)
memory usage: 41.8+ KB


In [197]:
#Vamos retirar as linhas duplicadas:
before = len(creditcard)
creditcard = creditcard.drop_duplicates()
after = len(creditcard)
print('Number of duplicate records dropped: ', str(before - after))
#Não possuímos linhas duplicadas nessa tabela CREDITCARD.

Number of duplicate records dropped:  0


In [198]:
#DISP_ID será a Foreign Key e a Primary Key será DISP_ID da tabela DISPOSITION.

In [199]:
#Vamos checar se realmente estão registrados somente 3 tipos de cartões.
set(creditcard['type'])
#Somente existem 3 categorias presentes mesmo.

{'classic', 'gold', 'junior'}

## Tabela ACCOUNT
As colunas ACCOUNT_ID e DISTRICT_ID serão necessárias.

In [200]:
account.info()
#Sem dados nulos na tabela.
#ACCOUNT_ID é do tipo int.
#DISTRICT_ID é do tipo int.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4500 entries, 0 to 4499
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   4500 non-null   int64 
 1   account_id   4500 non-null   int64 
 2   district_id  4500 non-null   int64 
 3   frequency    4500 non-null   object
 4   date         4500 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 210.9+ KB


In [201]:
#Vamos retirar as linhas duplicadas:
before = len(account)
account = account.drop_duplicates()
after = len(account)
print('Number of duplicate records dropped: ', str(before - after))
#Não possuímos linhas duplicadas nessa tabela CREDITCARD.

Number of duplicate records dropped:  0


In [202]:
#DISTRICT_ID será a Foreign Key e a Primary Key será DISTRICT_ID da tabela DEMOGRAPH.
#ACCOUNT_ID será a Primary Key e a Foreign Key será ACCOUNT_ID da tabela TRANSACTIONS.

## Tabela DEMOGRAPH
As colunas A1 ("DISTRICT_ID") e A3 (região) são necessárias.

In [203]:
demograph.info()
#Sem dados nulos na tabela.
#A1 é do tipo int.
#A3 é do tipo object/string.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  77 non-null     object
 1   A1          77 non-null     object
 2   A2          77 non-null     object
 3   A3          77 non-null     object
 4   A4          77 non-null     object
 5   A5          77 non-null     object
 6   A6          77 non-null     object
 7   A7          77 non-null     object
 8   A8          77 non-null     object
 9   A9          77 non-null     object
 10  A10         77 non-null     object
 11  A11         77 non-null     object
 12  A12         77 non-null     object
 13  A13         77 non-null     object
 14  A14         77 non-null     object
 15  A15         77 non-null     object
 16  A16         77 non-null     object
dtypes: object(17)
memory usage: 10.8+ KB


In [204]:
#Vamos retirar as linhas duplicadas:
before = len(demograph)
demograph = demograph.drop_duplicates()
after = len(demograph)
print('Number of duplicate records dropped: ', str(before - after))
#Não possuímos linhas duplicadas nessa tabela CREDITCARD.

Number of duplicate records dropped:  0


In [205]:
#DISTRICT_ID será a Foreign Key e a Primary Key será DISTRICT_ID da tabela DEMOGRAPH.
#ACCOUNT_ID será a Primary Key e a Foreign Key será ACCOUNT_ID da tabela TRANSACTIONS.

In [206]:
#A coluna de interesse é A3 que indica regiões.
set(demograph['A3'])
#Existem 8 regiões no total.

{'Prague',
 'central Bohemia',
 'east Bohemia',
 'north Bohemia',
 'north Moravia',
 'south Bohemia',
 'south Moravia',
 'west Bohemia'}

## Tabela CLIENT
As colunas CLIENT_ID e DISTRICT_ID (A1 na tabela DEMOGRAPH) são necessárias.

In [232]:
client.info()
#Sem dados nulos na tabela.
#CLIENT_ID é do tipo int.
#DISTRICT_ID é do tipo int.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Unnamed: 0    5369 non-null   int64
 1   client_id     5369 non-null   int64
 2   birth_number  5369 non-null   int64
 3   district_id   5369 non-null   int64
dtypes: int64(4)
memory usage: 167.9 KB


In [209]:
#Vamos retirar as linhas duplicadas:
before = len(client)
client = client.drop_duplicates()
after = len(client)
print('Number of duplicate records dropped: ', str(before - after))
#Não possuímos linhas duplicadas nessa tabela CLIENT.

Number of duplicate records dropped:  0


In [ ]:
#DISTRICT_ID será a Foreign Key e a Primary Key será DISTRICT_ID da tabela DEMOGRAPH.
#CLIENT_ID será a Primary Key e a Foreign Key será CLIENT_ID da tabela DISPOSITION.